Importing Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import numpy as np
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import minmax_scale
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier 
from sklearn import metrics
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.metrics import auc, plot_precision_recall_curve
import keras
from keras.datasets import mnist
from keras.models import Sequential,model_from_json
from keras.layers import Dense
from keras.optimizers import rmsprop_v2
import pylab as plt
from sklearn.utils.class_weight import compute_class_weight

Importing Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset_HTPred_classification.csv",na_values="?")
data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Circuit Name,No. of primary input gates,No. of primary output gates,No. of wires,No. of non-primary input gates,No. of non-primary output gates,No. of 3 or more input gates,sum CC0,sum CC1,sum CO,...,Obth L = 60,Obth L = 65,Obth L = 70,Obth L = 75,Obth L = 80,Obth L = 85,Obth L = 90,Obth L = 95,Obth L = 100,Class
0,AES_128_TjFree_bench.txt,257,128,970168,661048,316608,55232,1.879040e+07,2.583875e+07,1.062307e+09,...,2.744000e+03,2.401000e+03,2.058000e+03,1.715000e+03,1.372000e+03,1.029000e+03,6.860000e+02,3.430000e+02,0.0,Non-Trojan
1,b01.bench,2,2,120,85,45,8,4.337000e+03,2.640000e+03,1.723580e+05,...,2.570000e+03,2.248750e+03,1.927500e+03,1.606250e+03,1.285000e+03,9.637500e+02,6.425000e+02,3.212500e+02,0.0,Non-Trojan
2,b02.bench,1,1,67,48,26,4,7.799000e+03,4.656000e+03,1.764280e+05,...,3.725600e+03,3.259900e+03,2.794200e+03,2.328500e+03,1.862800e+03,1.397100e+03,9.314000e+02,4.657000e+02,0.0,Non-Trojan
3,b03.bench,4,4,424,284,152,20,5.520400e+04,3.799100e+04,5.664984e+06,...,2.083440e+04,1.823010e+04,1.562580e+04,1.302150e+04,1.041720e+04,7.812900e+03,5.208600e+03,2.604300e+03,0.0,Non-Trojan
4,b04.bench,11,8,1927,1333,718,53,7.510000e+13,4.150000e+13,3.720000e+15,...,3.900000e+12,3.410000e+12,2.920000e+12,2.440000e+12,1.950000e+12,1.460000e+12,9.750000e+11,4.870000e+11,0.0,Non-Trojan


Exploratory Data Analysis

In [ ]:
pandas_profiling.ProfileReport(data, minimal=True)

Output hidden; open in https://colab.research.google.com to view.

Data Preprocessing

In [ ]:
#Uisng Label Encoder
data_string = data.dtypes==object
data_cols = data.columns[data_string].tolist()
le = LabelEncoder()
data[data_cols] = data[data_cols].apply(lambda col: le.fit_transform(col))
data.head()

,Circuit Name,No. of primary input gates,No. of primary output gates,No. of wires,No. of non-primary input gates,No. of non-primary output gates,No. of 3 or more input gates,sum CC0,sum CC1,sum CO,...,Obth L = 60,Obth L = 65,Obth L = 70,Obth L = 75,Obth L = 80,Obth L = 85,Obth L = 90,Obth L = 95,Obth L = 100,Class
0,0,257,128,970168,661048,316608,55232,1.879040e+07,2.583875e+07,1.062307e+09,...,2.744000e+03,2.401000e+03,2.058000e+03,1.715000e+03,1.372000e+03,1.029000e+03,6.860000e+02,3.430000e+02,0.0,0
1,7,2,2,120,85,45,8,4.337000e+03,2.640000e+03,1.723580e+05,...,2.570000e+03,2.248750e+03,1.927500e+03,1.606250e+03,1.285000e+03,9.637500e+02,6.425000e+02,3.212500e+02,0.0,0
2,8,1,1,67,48,26,4,7.799000e+03,4.656000e+03,1.764280e+05,...,3.725600e+03,3.259900e+03,2.794200e+03,2.328500e+03,1.862800e+03,1.397100e+03,9.314000e+02,4.657000e+02,0.0,0
3,9,4,4,424,284,152,20,5.520400e+04,3.799100e+04,5.664984e+06,...,2.083440e+04,1.823010e+04,1.562580e+04,1.302150e+04,1.041720e+04,7.812900e+03,5.208600e+03,2.604300e+03,0.0,0
4,10,11,8,1927,1333,718,53,7.510000e+13,4.150000e+13,3.720000e+15,...,3.900000e+12,3.410000e+12,2.920000e+12,2.440000e+12,1.950000e+12,1.460000e+12,9.750000e+11,4.870000e+11,0.0,0


In [ ]:
X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train                                                    
                                    )

class_weights = dict(zip(np.unique(y_train), class_weights))
class_weights

{0: 3.23728813559322, 1: 0.5913312693498453}

Creating Deep Neural Networks

In [ ]:
# Deep Learning Model - All Feature Config 1
### Batch Size : 32
### Epochs: 24
### L1: 20    
### L2: 20
### L3: 20
### L4: 20
### L5: 20

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 32, epochs = 24, class_weight=class_weights)

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

CM = confusion_matrix(y_test, y_pred)
TP = CM[1][1]
FP = CM[0][1]

precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred)
prc = auc(recall, precision)

for _ in range(3):
  print()
print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("F1 Score: ", metrics.f1_score(y_test, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y_test, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y_test, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")

Epoch 1/24
12/12 [==============================] - 2s 7ms/step - loss: 0.6123 - accuracy: 0.7408
Epoch 2/24
12/12 [==============================] - 0s 6ms/step - loss: 0.4735 - accuracy: 0.8010
Epoch 3/24
12/12 [==============================] - 0s 4ms/step - loss: 0.3809 - accuracy: 0.8063
Epoch 4/24
12/12 [==============================] - 0s 7ms/step - loss: 0.3006 - accuracy: 0.8351
Epoch 5/24
12/12 [==============================] - 0s 4ms/step - loss: 0.2668 - accuracy: 0.9764
Epoch 6/24
12/12 [==============================] - 0s 4ms/step - loss: 0.2368 - accuracy: 0.9791
Epoch 7/24
12/12 [==============================] - 0s 5ms/step - loss: 0.2290 - accuracy: 0.9817
Epoch 8/24
12/12 [==============================] - 0s 6ms/step - loss: 0.1992 - accuracy: 0.9791
Epoch 9/24
12/12 [==============================] - 0s 5ms/step - loss: 0.1914 - accuracy: 0.9843
Epoch 10/24
12/12 [==============================] - 0s 6ms/step - loss: 0.1732 - accuracy: 0.9843
Epoch 11/24
12/12 [

In [ ]:
# Deep Learning Model - All Feature Config 2
### Batch Size : 32
### Epochs: 200
### L1: 20    
### L2: 20
### L3: 20
### L4: 20
### L5: 20

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 32, epochs = 200, class_weight=class_weights)

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

CM = confusion_matrix(y_test, y_pred)
TP = CM[1][1]
FP = CM[0][1]

precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred)
prc = auc(recall, precision)

for _ in range(3):
  print()
print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("F1 Score: ", metrics.f1_score(y_test, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y_test, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y_test, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")


Epoch 1/200
12/12 [==============================] - 1s 3ms/step - loss: 0.7474 - accuracy: 0.7408
Epoch 2/200
12/12 [==============================] - 0s 3ms/step - loss: 0.5706 - accuracy: 0.7827
Epoch 3/200
12/12 [==============================] - 0s 3ms/step - loss: 0.4648 - accuracy: 0.7853
Epoch 4/200
12/12 [==============================] - 0s 3ms/step - loss: 0.4148 - accuracy: 0.7984
Epoch 5/200
12/12 [==============================] - 0s 3ms/step - loss: 0.3679 - accuracy: 0.8037
Epoch 6/200
12/12 [==============================] - 0s 3ms/step - loss: 0.3222 - accuracy: 0.8063
Epoch 7/200
12/12 [==============================] - 0s 3ms/step - loss: 0.2868 - accuracy: 0.8063
Epoch 8/200
12/12 [==============================] - 0s 3ms/step - loss: 0.2454 - accuracy: 0.8455
Epoch 9/200
12/12 [==============================] - 0s 3ms/step - loss: 0.2122 - accuracy: 0.9817
Epoch 10/200
12/12 [==============================] - 0s 3ms/step - loss: 0.1850 - accuracy: 0.9843
Epoch 11/

In [ ]:
# Deep Learning Model - All Feature Config 3
### Batch Size : 12
### Epochs: 24
### L1: 2    
### L2: 20
### L3: 200
### L4: 2
### L5: 20

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=200, activation='relu'))
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 12, epochs = 24, class_weight=class_weights)

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

CM = confusion_matrix(y_test, y_pred)
TP = CM[1][1]
FP = CM[0][1]

precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred)
prc = auc(recall, precision)

for _ in range(3):
  print()
print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("F1 Score: ", metrics.f1_score(y_test, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y_test, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y_test, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")


Epoch 1/24
32/32 [==============================] - 1s 3ms/step - loss: 0.6953 - accuracy: 0.8141
Epoch 2/24
32/32 [==============================] - 0s 3ms/step - loss: 0.6771 - accuracy: 0.7723
Epoch 3/24
32/32 [==============================] - 0s 3ms/step - loss: 0.5550 - accuracy: 0.8010
Epoch 4/24
32/32 [==============================] - 0s 3ms/step - loss: 0.4645 - accuracy: 0.8403
Epoch 5/24
32/32 [==============================] - 0s 3ms/step - loss: 0.4099 - accuracy: 0.9372
Epoch 6/24
32/32 [==============================] - 0s 3ms/step - loss: 0.3843 - accuracy: 0.8901
Epoch 7/24
32/32 [==============================] - 0s 3ms/step - loss: 0.3670 - accuracy: 0.9607
Epoch 8/24
32/32 [==============================] - 0s 3ms/step - loss: 0.3501 - accuracy: 0.9843
Epoch 9/24
32/32 [==============================] - 0s 3ms/step - loss: 0.3345 - accuracy: 0.9503
Epoch 10/24
32/32 [==============================] - 0s 3ms/step - loss: 0.3234 - accuracy: 0.9817
Epoch 11/24
32/32 [

In [ ]:
# Deep Learning Model - All Feature Config 4
### Batch Size : 12
### Epochs: 24
### L1: 2    
### L2: 20
### L3: 200
### L4: 400
### L5: 2

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=200, activation='relu'))
ann.add(tf.keras.layers.Dense(units=400, activation='relu'))
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 12, epochs = 24, class_weight=class_weights)

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

CM = confusion_matrix(y_test, y_pred)
TP = CM[1][1]
FP = CM[0][1]

precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred)
prc = auc(recall, precision)

for _ in range(3):
  print()
print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("F1 Score: ", metrics.f1_score(y_test, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y_test, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y_test, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")


Epoch 1/24
32/32 [==============================] - 1s 6ms/step - loss: 0.6619 - accuracy: 0.8298
Epoch 2/24
32/32 [==============================] - 0s 6ms/step - loss: 0.5772 - accuracy: 0.8141
Epoch 3/24
32/32 [==============================] - 0s 4ms/step - loss: 0.5481 - accuracy: 0.7932
Epoch 4/24
32/32 [==============================] - 0s 4ms/step - loss: 0.5353 - accuracy: 0.7906
Epoch 5/24
32/32 [==============================] - 0s 4ms/step - loss: 0.5127 - accuracy: 0.7932
Epoch 6/24
32/32 [==============================] - 0s 4ms/step - loss: 0.4947 - accuracy: 0.8063
Epoch 7/24
32/32 [==============================] - 0s 4ms/step - loss: 0.4939 - accuracy: 0.8063
Epoch 8/24
32/32 [==============================] - 0s 4ms/step - loss: 0.5011 - accuracy: 0.8089
Epoch 9/24
32/32 [==============================] - 0s 5ms/step - loss: 0.4642 - accuracy: 0.8089
Epoch 10/24
32/32 [==============================] - 0s 4ms/step - loss: 0.4826 - accuracy: 0.8063
Epoch 11/24
32/32 [

In [ ]:
# Deep Learning Model - All Feature Config 5
### Batch Size : 12
### Epochs: 200
### L1: 2    
### L2: 20
### L3: 200
### L4: 400
### L5: 2

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=200, activation='relu'))
ann.add(tf.keras.layers.Dense(units=400, activation='relu'))
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 12, epochs = 200, class_weight=class_weights)

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

CM = confusion_matrix(y_test, y_pred)
TP = CM[1][1]
FP = CM[0][1]

precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred)
prc = auc(recall, precision)

for _ in range(3):
  print()
print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("F1 Score: ", metrics.f1_score(y_test, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y_test, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y_test, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")


Epoch 1/200
32/32 [==============================] - 1s 6ms/step - loss: 0.7169 - accuracy: 0.7565
Epoch 2/200
32/32 [==============================] - 0s 6ms/step - loss: 0.6311 - accuracy: 0.8010
Epoch 3/200
32/32 [==============================] - 0s 6ms/step - loss: 0.5826 - accuracy: 0.7906
Epoch 4/200
32/32 [==============================] - 0s 7ms/step - loss: 0.5197 - accuracy: 0.7984
Epoch 5/200
32/32 [==============================] - 0s 6ms/step - loss: 0.4270 - accuracy: 0.8063
Epoch 6/200
32/32 [==============================] - 0s 5ms/step - loss: 0.3693 - accuracy: 0.8168
Epoch 7/200
32/32 [==============================] - 0s 4ms/step - loss: 0.4051 - accuracy: 0.7408
Epoch 8/200
32/32 [==============================] - 0s 4ms/step - loss: 0.3653 - accuracy: 0.8037
Epoch 9/200
32/32 [==============================] - 0s 4ms/step - loss: 0.2962 - accuracy: 0.8141
Epoch 10/200
32/32 [==============================] - 0s 4ms/step - loss: 0.2474 - accuracy: 0.8194
Epoch 11/

Extracting Top 50 Features based on Information Gain

In [ ]:
listTop50 = [
'No. of 3 or more input gates',
'No. of XOR',
'P0 >= 0.9',
'P0 >= 0.8',
'p0 >= 0.4',
'p0 >= 0.3',
'p0 >= 0.2',
'p0 >= 0.1',
'P1 >= 0.6',
'SUM P0 * SUM C0',
'NORMALISED P0',
'NORMALISED P1',
'NORMALISED CC0',
'HARMONIC MEAN CC1',
'SUM CC0*SUM P1',
'SUM CC0 * SUM CC1 * SUM CO',
'Harmonic Mean CCS',
'Geometric Mean CCS',
'Harmonic Mean fan_in_1',
'Geometric Mean fan_in_1',
'fan_in_1 1 - 5',
'Geometric Mean fan_in_2',
'fan_in_2 5 - 10',
'Average fan_in_3',
'Normalised fan_in_3',
'Geometric Mean fan_in_3',
'fan_in_3 1 - 5',
'Average fan_in_4',
'Normalised fan_in_4',
'Standard Deviation fan_in_4',
'Population Standard Deviation fan_in_4',
'Population Variance fan_in_4',
'Variance fan_in_4',
'Geometric Mean fan_in_4',
'Average fan_in_5',
'Normalised fan_in_5',
'Standard Deviation fan_in_5',
'Population Standard Deviation fan_in_5',
'Population Variance fan_in_5',
'Variance fan_in_5',
'Geometric Mean fan_in_5',
'Harmonic Mean in_nearest_pin',
'in_nearest_pin 1 - 5',
'out_nearest_pout 5 - 10',
"Variance in_ff_1",
"Sum in_ff_4",
"Sum in_nearest_ff",
"in_nearest_ff 1 - 5",
"in_nearest_ff > 100",
"Sum out_nearest_ff",
'Class'
]

In [ ]:
import pandas as pd

for name, values in data.iteritems():
  if(name.strip() not in listTop50):
    if(name.strip() == "Class"):
      print("Class Popped")
    data.pop(name)

X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

data.to_csv('top_50_features.csv', index = False, encoding='utf-8')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [ ]:
data.head()

,No. of 3 or more input gates,No. of XOR,P0 >= 0.9,P0 >= 0.8,p0 >= 0.4,p0 >= 0.3,p0 >= 0.2,p0 >= 0.1,P1 >= 0.6,SUM P0 * SUM C0,...,Harmonic Mean in_nearest_pin,in_nearest_pin 1 - 5,out_nearest_pout 5 - 10,Variance in_ff_1,Sum in_ff_4,Sum in_nearest_ff,in_nearest_ff 1 - 5,in_nearest_ff > 100,Sum out_nearest_ff,Class
0,55232,1728,117136,144528,202584,205024,219408,242736,114024,3.430000e+12,...,2.992876,286344,10932,0.020775,373576,1378520,307640,640,1575520,0
1,8,0,0,1,22,26,37,43,23,7.702381e+04,...,2.431181,34,18,0.101010,31,10050,30,10,2070,0
2,4,0,0,1,12,18,24,26,14,8.086893e+04,...,2.071713,20,14,0.135385,19,1029,21,1,1039,0
3,20,0,0,0,79,80,122,144,73,3.171978e+06,...,5.792680,26,48,0.159463,200,169,122,0,235,0
4,53,0,22,26,333,382,590,652,385,2.010000e+16,...,2.652446,460,330,0.083589,450,27925,491,26,2872,0


Testing Multiple Configurations of Deep Neural Network for the extracted features

In [ ]:
# Deep Learning Model - Top 50 Config 1
### Batch Size : 32
### Epochs: 24
### L1: 20    
### L2: 20
### L3: 20
### L4: 20
### L5: 20

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 32, epochs = 24, class_weight=class_weights)

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

CM = confusion_matrix(y_test, y_pred)
TP = CM[1][1]
FP = CM[0][1]

precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred)
prc = auc(recall, precision)

for _ in range(3):
  print()
print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("F1 Score: ", metrics.f1_score(y_test, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y_test, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y_test, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")



Epoch 1/24
12/12 [==============================] - 2s 5ms/step - loss: 0.6335 - accuracy: 0.1545
Epoch 2/24
12/12 [==============================] - 0s 6ms/step - loss: 0.5785 - accuracy: 0.1885
Epoch 3/24
12/12 [==============================] - 0s 5ms/step - loss: 0.5352 - accuracy: 0.6021
Epoch 4/24
12/12 [==============================] - 0s 5ms/step - loss: 0.5003 - accuracy: 0.6047
Epoch 5/24
12/12 [==============================] - 0s 5ms/step - loss: 0.4731 - accuracy: 0.7330
Epoch 6/24
12/12 [==============================] - 0s 6ms/step - loss: 0.4524 - accuracy: 0.8717
Epoch 7/24
12/12 [==============================] - 0s 5ms/step - loss: 0.4341 - accuracy: 0.8822
Epoch 8/24
12/12 [==============================] - 0s 5ms/step - loss: 0.4170 - accuracy: 0.8822
Epoch 9/24
12/12 [==============================] - 0s 4ms/step - loss: 0.3978 - accuracy: 0.8874
Epoch 10/24
12/12 [==============================] - 0s 4ms/step - loss: 0.3817 - accuracy: 0.9372
Epoch 11/24
12/12 [

In [ ]:
# Deep Learning Model - Top 50 Config 2
### Batch Size : 32
### Epochs: 200
### L1: 20 
### L2: 20
### L3: 20
### L4: 20
### L5: 20

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 32, epochs = 200, class_weight=class_weights)

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

CM = confusion_matrix(y_test, y_pred)
TP = CM[1][1]
FP = CM[0][1]
FN = CM[1][0]
TN = CM[0][0]

TPR = TP / (TP + FN)
FPR = FP / (FP + TN)

precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred)
prc = auc(recall, precision)

for _ in range(3):
  print()

print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("TP Rate: ", TPR)
print("FP Rate: ", FPR)
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("F1 Score: ", metrics.f1_score(y_test, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y_test, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y_test, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")


Epoch 1/200
12/12 [==============================] - 1s 3ms/step - loss: 0.6821 - accuracy: 0.3613
Epoch 2/200
12/12 [==============================] - 0s 3ms/step - loss: 0.6344 - accuracy: 0.3927
Epoch 3/200
12/12 [==============================] - 0s 3ms/step - loss: 0.6004 - accuracy: 0.4372
Epoch 4/200
12/12 [==============================] - 0s 4ms/step - loss: 0.5589 - accuracy: 0.6073
Epoch 5/200
12/12 [==============================] - 0s 3ms/step - loss: 0.5179 - accuracy: 0.8010
Epoch 6/200
12/12 [==============================] - 0s 4ms/step - loss: 0.4789 - accuracy: 0.8010
Epoch 7/200
12/12 [==============================] - 0s 3ms/step - loss: 0.4408 - accuracy: 0.8508
Epoch 8/200
12/12 [==============================] - 0s 3ms/step - loss: 0.3996 - accuracy: 0.8848
Epoch 9/200
12/12 [==============================] - 0s 3ms/step - loss: 0.3645 - accuracy: 0.8927
Epoch 10/200
12/12 [==============================] - 0s 3ms/step - loss: 0.3372 - accuracy: 0.8901
Epoch 11/

In [ ]:
# Deep Learning Model - Top 50 Config 3
### Batch Size : 12
### Epochs: 24
### L1: 2    
### L2: 20
### L3: 200
### L4: 10
### L5: 20

import pickle

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=200, activation='relu'))
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 12, epochs = 24, class_weight=class_weights)

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

filename = 'finalized_model.sav'
pickle.dump(ann, open(filename, 'wb'))

CM = confusion_matrix(y_test, y_pred)
TP = CM[1][1]
FP = CM[0][1]

precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred)
prc = auc(recall, precision)

for _ in range(3):
  print()
print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("F1 Score: ", metrics.f1_score(y_test, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y_test, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y_test, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")

Epoch 1/24
32/32 [==============================] - 1s 2ms/step - loss: 0.6786 - accuracy: 0.2094
Epoch 2/24
32/32 [==============================] - 0s 2ms/step - loss: 0.6265 - accuracy: 0.3822
Epoch 3/24
32/32 [==============================] - 0s 2ms/step - loss: 0.5608 - accuracy: 0.6126
Epoch 4/24
32/32 [==============================] - 0s 2ms/step - loss: 0.4812 - accuracy: 0.8168
Epoch 5/24
32/32 [==============================] - 0s 2ms/step - loss: 0.4120 - accuracy: 0.8743
Epoch 6/24
32/32 [==============================] - 0s 2ms/step - loss: 0.3858 - accuracy: 0.8691
Epoch 7/24
32/32 [==============================] - 0s 2ms/step - loss: 0.3567 - accuracy: 0.8901
Epoch 8/24
32/32 [==============================] - 0s 2ms/step - loss: 0.3375 - accuracy: 0.8822
Epoch 9/24
32/32 [==============================] - 0s 2ms/step - loss: 0.3303 - accuracy: 0.8822
Epoch 10/24
32/32 [==============================] - 0s 2ms/step - loss: 0.3209 - accuracy: 0.8901
Epoch 11/24
32/32 [

In [ ]:
#Load Model and create predictions

data_new = pd.read_csv (r'top_50_features.csv')
data_new.head()

X = data_new.iloc[:, 1:-1].values
y = data_new.iloc[:, -1].values

sc = StandardScaler()
X = sc.fit_transform(X)

loaded_model = pickle.load(open(filename, 'rb'))
y_pred = loaded_model.predict(X)

y_pred = (y_pred > 0.5)

list_of_tuples = list(zip(y_pred, y_test))
 
# Converting lists of tuples into
# pandas Dataframe.

data_new["Prediction"] = y_pred

data_new.replace({False: "Non Trojan", True: "Trojan"}, inplace=True)

# data_new['Prediction'] = data_new['Prediction'].map({0:'No Trojan', 1:'Trojan'})

data_new.to_csv('result.csv', index = False, encoding='utf-8')

print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y, y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("Precision: ", metrics.precision_score(y, y_pred))
print("Recall: ", metrics.recall_score(y, y_pred))
print("F1 Score: ", metrics.f1_score(y, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")

# # print("Enter the Label (Trojan = 1, No Trojan = 0)")



# data_new
# data = {'Prediction': y_pred,}
# fdf = pd.DataFrame(data, )
# fdf.head()
# print(loaded_model.evaluate(X_train, y_train))

,No. of 3 or more input gates,No. of XOR,P0 >= 0.9,P0 >= 0.8,p0 >= 0.4,p0 >= 0.3,p0 >= 0.2,p0 >= 0.1,P1 >= 0.6,SUM P0 * SUM C0,...,in_nearest_pin 1 - 5,out_nearest_pout 5 - 10,Variance in_ff_1,Sum in_ff_4,Sum in_nearest_ff,in_nearest_ff 1 - 5,in_nearest_ff > 100,Sum out_nearest_ff,Class,Prediction
0,55232,1728,117136,144528,202584,205024,219408,242736,114024,3.430000e+12,...,286344,10932,0.020775,373576,1378520,307640,640,1575520,0,Trojan
1,8,0,0,1,22,26,37,43,23,7.702381e+04,...,34,18,0.101010,31,10050,30,10,2070,0,Non Trojan
2,4,0,0,1,12,18,24,26,14,8.086893e+04,...,20,14,0.135385,19,1029,21,1,1039,0,Non Trojan
3,20,0,0,0,79,80,122,144,73,3.171978e+06,...,26,48,0.159463,200,169,122,0,235,0,Non Trojan
4,53,0,22,26,333,382,590,652,385,2.010000e+16,...,460,330,0.083589,450,27925,491,26,2872,0,Non Trojan


In [ ]:
# Deep Learning Model - Top 50 Config 4
### Batch Size : 12
### Epochs: 24
### L1: 2    
### L2: 20
### L3: 200
### L4: 400
### L5: 2

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=200, activation='relu'))
ann.add(tf.keras.layers.Dense(units=400, activation='relu'))
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 12, epochs = 24, class_weight=class_weights)

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

CM = confusion_matrix(y_test, y_pred)
TP = CM[1][1]
FP = CM[0][1]

precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred)
prc = auc(recall, precision)

for _ in range(3):
  print()
print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("F1 Score: ", metrics.f1_score(y_test, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y_test, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y_test, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")

Epoch 1/24
32/32 [==============================] - 1s 4ms/step - loss: 0.6622 - accuracy: 0.2539
Epoch 2/24
32/32 [==============================] - 0s 4ms/step - loss: 0.6125 - accuracy: 0.4162
Epoch 3/24
32/32 [==============================] - 0s 4ms/step - loss: 0.5539 - accuracy: 0.6990
Epoch 4/24
32/32 [==============================] - 0s 4ms/step - loss: 0.4864 - accuracy: 0.8220
Epoch 5/24
32/32 [==============================] - 0s 4ms/step - loss: 0.4466 - accuracy: 0.8848
Epoch 6/24
32/32 [==============================] - 0s 4ms/step - loss: 0.4322 - accuracy: 0.8717
Epoch 7/24
32/32 [==============================] - 0s 4ms/step - loss: 0.4103 - accuracy: 0.9581
Epoch 8/24
32/32 [==============================] - 0s 4ms/step - loss: 0.4109 - accuracy: 0.9110
Epoch 9/24
32/32 [==============================] - 0s 4ms/step - loss: 0.4008 - accuracy: 0.9424
Epoch 10/24
32/32 [==============================] - 0s 4ms/step - loss: 0.3917 - accuracy: 0.9607
Epoch 11/24
32/32 [

In [ ]:
# Deep Learning Model - Top 50 Config 5
### Batch Size : 12
### Epochs: 200
### L1: 2    
### L2: 20
### L3: 200
### L4: 400
### L5: 2

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=200, activation='relu'))
ann.add(tf.keras.layers.Dense(units=400, activation='relu'))
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 12, epochs = 200, class_weight=class_weights)

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

CM = confusion_matrix(y_test, y_pred)
TP = CM[1][1]
FP = CM[0][1]

precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred)
prc = auc(recall, precision)

for _ in range(3):
  print()
print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("F1 Score: ", metrics.f1_score(y_test, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y_test, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y_test, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")

Epoch 1/200
32/32 [==============================] - 1s 8ms/step - loss: 0.7030 - accuracy: 0.8455
Epoch 2/200
32/32 [==============================] - 0s 8ms/step - loss: 0.6909 - accuracy: 0.8455
Epoch 3/200
32/32 [==============================] - 0s 8ms/step - loss: 0.6867 - accuracy: 0.8403
Epoch 4/200
32/32 [==============================] - 0s 11ms/step - loss: 0.6766 - accuracy: 0.8534
Epoch 5/200
32/32 [==============================] - 0s 9ms/step - loss: 0.6689 - accuracy: 0.8115
Epoch 6/200
32/32 [==============================] - 0s 6ms/step - loss: 0.6522 - accuracy: 0.8770
Epoch 7/200
32/32 [==============================] - 0s 6ms/step - loss: 0.6264 - accuracy: 0.8927
Epoch 8/200
32/32 [==============================] - 0s 6ms/step - loss: 0.6069 - accuracy: 0.9241
Epoch 9/200
32/32 [==============================] - 0s 8ms/step - loss: 0.6043 - accuracy: 0.9372
Epoch 10/200
32/32 [==============================] - 0s 7ms/step - loss: 0.5864 - accuracy: 0.9005
Epoch 11

Extracting Top 45 Features based on Information Gain

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset_HTPred_classification.csv",na_values="?")
data_string = data.dtypes==object
data_cols = data.columns[data_string].tolist()
le = LabelEncoder()
data[data_cols] = data[data_cols].apply(lambda col: le.fit_transform(col))

In [ ]:
listTop45 = [
'No. of 3 or more input gates',
'No. of XOR',
'P0 >= 0.9',
'P0 >= 0.8',
'p0 >= 0.4',
'p0 >= 0.3',
'p0 >= 0.2',
'p0 >= 0.1',
'P1 >= 0.6',
'SUM P0 * SUM C0',
'NORMALISED P0',
'NORMALISED P1',
'NORMALISED CC0',
'HARMONIC MEAN CC1',
'SUM CC0*SUM P1',
'SUM CC0 * SUM CC1 * SUM CO',
'Harmonic Mean CCS',
'Geometric Mean CCS',
'Harmonic Mean fan_in_1',
'Geometric Mean fan_in_1',
'fan_in_1 1 - 5',
'Geometric Mean fan_in_2',
'fan_in_2 5 - 10',
'Average fan_in_3',
'Normalised fan_in_3',
'Geometric Mean fan_in_3',
'fan_in_3 1 - 5',
'Average fan_in_4',
'Normalised fan_in_4',
'Standard Deviation fan_in_4',
'Population Standard Deviation fan_in_4',
'Population Variance fan_in_4',
'Variance fan_in_4',
'Geometric Mean fan_in_4',
'Average fan_in_5',
'Normalised fan_in_5',
'Standard Deviation fan_in_5',
'Population Standard Deviation fan_in_5',
'Population Variance fan_in_5',
'Variance fan_in_5',
'Geometric Mean fan_in_5',
'Harmonic Mean in_nearest_pin',
'in_nearest_pin 1 - 5',
'out_nearest_pout 5 - 10',
"Variance in_ff_1",
'Class'
]

In [ ]:
for name, values in data.iteritems():
  if(name.strip() not in listTop45):
    if(name.strip() == "Class"):
      print("Class Popped")
    data.pop(name)

X = data.iloc[:, 1:-1].values
y = data.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

data.head()

,No. of 3 or more input gates,No. of XOR,P0 >= 0.9,P0 >= 0.8,p0 >= 0.4,p0 >= 0.3,p0 >= 0.2,p0 >= 0.1,P1 >= 0.6,SUM P0 * SUM C0,...,Standard Deviation fan_in_5,Population Standard Deviation fan_in_5,Population Variance fan_in_5,Variance fan_in_5,Geometric Mean fan_in_5,Harmonic Mean in_nearest_pin,in_nearest_pin 1 - 5,out_nearest_pout 5 - 10,Variance in_ff_1,Class
0,55232,1728,117136,144528,202584,205024,219408,242736,114024,3.430000e+12,...,14.057330,14.057308,197.607913,197.608537,13.613545,2.992876,286344,10932,0.020775,0
1,8,0,0,1,22,26,37,43,23,7.702381e+04,...,9.395733,9.290749,86.318025,88.279798,14.020861,2.431181,34,18,0.101010,0
2,4,0,0,1,12,18,24,26,14,8.086893e+04,...,4.134657,4.054364,16.437870,17.095385,8.812276,2.071713,20,14,0.135385,0
3,20,0,0,0,79,80,122,144,73,3.171978e+06,...,10.221800,10.188120,103.797784,104.485186,10.802326,5.792680,26,48,0.159463,0
4,53,0,22,26,333,382,590,652,385,2.010000e+16,...,10.765379,10.757880,115.731978,115.893389,10.545540,2.652446,460,330,0.083589,0


In [ ]:
# Deep Learning Model - Top 50 Config 1
### Batch Size : 32
### Epochs: 24
### L1: 20    
### L2: 20
### L3: 20
### L4: 20
### L5: 20

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 32, epochs = 24, class_weight=class_weights)

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

CM = confusion_matrix(y_test, y_pred)
TP = CM[1][1]
FP = CM[0][1]

precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred)
prc = auc(recall, precision)

for _ in range(3):
  print()
print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("F1 Score: ", metrics.f1_score(y_test, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y_test, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y_test, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")



Epoch 1/24
12/12 [==============================] - 1s 3ms/step - loss: 0.6715 - accuracy: 0.2618
Epoch 2/24
12/12 [==============================] - 0s 3ms/step - loss: 0.6345 - accuracy: 0.2408
Epoch 3/24
12/12 [==============================] - 0s 3ms/step - loss: 0.5992 - accuracy: 0.3298
Epoch 4/24
12/12 [==============================] - 0s 3ms/step - loss: 0.5663 - accuracy: 0.4529
Epoch 5/24
12/12 [==============================] - 0s 3ms/step - loss: 0.5299 - accuracy: 0.7932
Epoch 6/24
12/12 [==============================] - 0s 2ms/step - loss: 0.4915 - accuracy: 0.8927
Epoch 7/24
12/12 [==============================] - 0s 3ms/step - loss: 0.4562 - accuracy: 0.8901
Epoch 8/24
12/12 [==============================] - 0s 3ms/step - loss: 0.4178 - accuracy: 0.8874
Epoch 9/24
12/12 [==============================] - 0s 3ms/step - loss: 0.3859 - accuracy: 0.8927
Epoch 10/24
12/12 [==============================] - 0s 3ms/step - loss: 0.3563 - accuracy: 0.9005
Epoch 11/24
12/12 [

In [ ]:
# Deep Learning Model - Top 50 Config 2
### Batch Size : 32
### Epochs: 200
### L1: 20 
### L2: 20
### L3: 20
### L4: 20
### L5: 20

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 32, epochs = 200, class_weight=class_weights)

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

CM = confusion_matrix(y_test, y_pred)
TP = CM[1][1]
FP = CM[0][1]

precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred)
prc = auc(recall, precision)

for _ in range(3):
  print()
print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("F1 Score: ", metrics.f1_score(y_test, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y_test, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y_test, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")



Epoch 1/200
12/12 [==============================] - 1s 7ms/step - loss: 0.6999 - accuracy: 0.8691
Epoch 2/200
12/12 [==============================] - 0s 6ms/step - loss: 0.6593 - accuracy: 0.8037
Epoch 3/200
12/12 [==============================] - 0s 4ms/step - loss: 0.6377 - accuracy: 0.6571
Epoch 4/200
12/12 [==============================] - 0s 5ms/step - loss: 0.6172 - accuracy: 0.6283
Epoch 5/200
12/12 [==============================] - 0s 5ms/step - loss: 0.5910 - accuracy: 0.8272
Epoch 6/200
12/12 [==============================] - 0s 8ms/step - loss: 0.5602 - accuracy: 0.8770
Epoch 7/200
12/12 [==============================] - 0s 7ms/step - loss: 0.5091 - accuracy: 0.8743
Epoch 8/200
12/12 [==============================] - 0s 9ms/step - loss: 0.4611 - accuracy: 0.8717
Epoch 9/200
12/12 [==============================] - 0s 4ms/step - loss: 0.4064 - accuracy: 0.8743
Epoch 10/200
12/12 [==============================] - 0s 6ms/step - loss: 0.3851 - accuracy: 0.8848
Epoch 11/

In [ ]:
# Deep Learning Model - Top 50 Config 3
### Batch Size : 12
### Epochs: 24
### L1: 2    
### L2: 20
### L3: 200
### L4: 10
### L5: 20

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=200, activation='relu'))
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 12, epochs = 24, class_weight=class_weights)

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

CM = confusion_matrix(y_test, y_pred)
TP = CM[1][1]
FP = CM[0][1]

precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred)
prc = auc(recall, precision)

for _ in range(3):
  print()
print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("F1 Score: ", metrics.f1_score(y_test, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y_test, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y_test, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")

Epoch 1/24
32/32 [==============================] - 1s 3ms/step - loss: 0.6869 - accuracy: 0.5393
Epoch 2/24
32/32 [==============================] - 0s 3ms/step - loss: 0.6651 - accuracy: 0.6990
Epoch 3/24
32/32 [==============================] - 0s 3ms/step - loss: 0.6409 - accuracy: 0.8220
Epoch 4/24
32/32 [==============================] - 0s 3ms/step - loss: 0.6145 - accuracy: 0.8272
Epoch 5/24
32/32 [==============================] - 0s 3ms/step - loss: 0.5831 - accuracy: 0.8953
Epoch 6/24
32/32 [==============================] - 0s 3ms/step - loss: 0.5390 - accuracy: 0.9188
Epoch 7/24
32/32 [==============================] - 0s 3ms/step - loss: 0.5088 - accuracy: 0.9005
Epoch 8/24
32/32 [==============================] - 0s 3ms/step - loss: 0.4699 - accuracy: 0.9241
Epoch 9/24
32/32 [==============================] - 0s 4ms/step - loss: 0.4329 - accuracy: 0.9241
Epoch 10/24
32/32 [==============================] - 0s 3ms/step - loss: 0.4152 - accuracy: 0.9398
Epoch 11/24
32/32 [

In [ ]:
# Deep Learning Model - Top 50 Config 4
### Batch Size : 12
### Epochs: 24
### L1: 2    
### L2: 20
### L3: 200
### L4: 400
### L5: 2

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=200, activation='relu'))
ann.add(tf.keras.layers.Dense(units=400, activation='relu'))
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 12, epochs = 24, class_weight=class_weights)

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

CM = confusion_matrix(y_test, y_pred)
TP = CM[1][1]
FP = CM[0][1]

precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred)
prc = auc(recall, precision)

for _ in range(3):
  print()
print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("F1 Score: ", metrics.f1_score(y_test, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y_test, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y_test, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")

Epoch 1/24
32/32 [==============================] - 1s 4ms/step - loss: 0.7215 - accuracy: 0.5340
Epoch 2/24
32/32 [==============================] - 0s 4ms/step - loss: 0.6895 - accuracy: 0.3796
Epoch 3/24
32/32 [==============================] - 0s 4ms/step - loss: 0.6787 - accuracy: 0.6361
Epoch 4/24
32/32 [==============================] - 0s 4ms/step - loss: 0.6487 - accuracy: 0.7670
Epoch 5/24
32/32 [==============================] - 0s 4ms/step - loss: 0.6195 - accuracy: 0.7801
Epoch 6/24
32/32 [==============================] - 0s 4ms/step - loss: 0.6027 - accuracy: 0.7853
Epoch 7/24
32/32 [==============================] - 0s 4ms/step - loss: 0.5805 - accuracy: 0.7958
Epoch 8/24
32/32 [==============================] - 0s 4ms/step - loss: 0.5712 - accuracy: 0.7932
Epoch 9/24
32/32 [==============================] - 0s 4ms/step - loss: 0.5644 - accuracy: 0.7958
Epoch 10/24
32/32 [==============================] - 0s 4ms/step - loss: 0.5615 - accuracy: 0.7958
Epoch 11/24
32/32 [

In [ ]:
# Deep Learning Model - Top 50 Config 5
### Batch Size : 12
### Epochs: 200
### L1: 2    
### L2: 20
### L3: 200
### L4: 400
### L5: 2

ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(tf.keras.layers.Dense(units=200, activation='relu'))
ann.add(tf.keras.layers.Dense(units=400, activation='relu'))
ann.add(tf.keras.layers.Dense(units=2, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(X_train, y_train, batch_size = 12, epochs = 200, class_weight=class_weights)

y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)

CM = confusion_matrix(y_test, y_pred)
TP = CM[1][1]
FP = CM[0][1]

precision, recall, thresholds = metrics.precision_recall_curve(y_test, y_pred)
prc = auc(recall, precision)

for _ in range(3):
  print()
print("-----------------------------------------------------------------------------")
print("Accuracy: ",metrics.accuracy_score(y_test, y_pred))
print("TP Rate: ", TP)
print("FP Rate: ", FP)
print("Precision: ", metrics.precision_score(y_test, y_pred))
print("Recall: ", metrics.recall_score(y_test, y_pred))
print("F1 Score: ", metrics.f1_score(y_test, y_pred))
print("MCC: ", metrics.matthews_corrcoef(y_test, y_pred))
print("ROC Area: ", metrics.roc_auc_score(y_test, y_pred))
print("PRC Area: ", prc)
print("-----------------------------------------------------------------------------")

Epoch 1/200
32/32 [==============================] - 1s 3ms/step - loss: 0.6942 - accuracy: 0.8403
Epoch 2/200
32/32 [==============================] - 0s 4ms/step - loss: 0.6952 - accuracy: 0.8455
Epoch 3/200
32/32 [==============================] - 0s 4ms/step - loss: 0.6942 - accuracy: 0.5576
Epoch 4/200
32/32 [==============================] - 0s 5ms/step - loss: 0.6932 - accuracy: 0.1649
Epoch 5/200
32/32 [==============================] - 0s 4ms/step - loss: 0.6932 - accuracy: 0.1545
Epoch 6/200
32/32 [==============================] - 0s 4ms/step - loss: 0.6932 - accuracy: 0.3482
Epoch 7/200
32/32 [==============================] - 0s 4ms/step - loss: 0.6933 - accuracy: 0.1545
Epoch 8/200
32/32 [==============================] - 0s 4ms/step - loss: 0.6932 - accuracy: 0.2016
Epoch 9/200
32/32 [==============================] - 0s 4ms/step - loss: 0.6933 - accuracy: 0.1545
Epoch 10/200
32/32 [==============================] - 0s 4ms/step - loss: 0.6932 - accuracy: 0.1545
Epoch 11/